In [89]:
import requests
from collections import Counter
import os
from pathlib import Path
import json
import pandas as pd
import librosa

In [90]:
audios_path = Path('audios_pedro')
tags_path = Path('etiquetas_pedro')

In [92]:
FUSA_SERVER="https://api.labacam.org"
#FUSA_SERVER="http://localhost:8080"
API_KEY="zIkYk2VwBiC762O4yFSd"

ENDPOINT="predictions"
MODEL="PANN-tag-5s"

URI = f"{FUSA_SERVER}/{ENDPOINT}?model={MODEL}/"
#URI = f"{FUSA_SERVER}/{ENDPOINT}/{MODEL}/{VERSION}/"

print(URI)

https://api.labacam.org/predictions?model=PANN-tag-5s/


In [4]:
threshold = 0.01

In [5]:
def filter_predictions(categories_dict, threshold):
    filtered_dict = {}
    for key, values in categories_dict.items():
        values_list = []
        for value in values:
            if value['probability'] > threshold:
                values_list.append(value)
            filtered_dict.update({key:values_list})
    return filtered_dict

def calculate_metrics(filtered_predictions, audio_duration):
    df = pd.DataFrame([], columns  = ['event', 'duration_ratio', 'avg_acc'])
    class_tags, duration_ratio, avg_acc = [], [], []
    for class_tags in filtered_predictions:
        avg_acc = 0
        tags = filtered_predictions[class_tags]
        c = len(tags)
        if c > 0:
            for tag in tags:
                avg_acc += tag['probability']
            duration_ratio = (c * 5) / audio_duration
            avg_acc = avg_acc / c
            d = {'event' : class_tags, 'duration_ratio': round(duration_ratio, 3), 'avg_acc': round(avg_acc, 3)}
            output = pd.DataFrame([d])
            df = pd.concat([df, output], ignore_index=True)
    df = df.sort_values(by='duration_ratio', ascending=False).head(5)
    return df

def get_audio_duration(audio_path):
    return librosa.get_duration(filename=audio_path)

In [115]:
c = 0
for root, dirs, files in os.walk(audios_path):
    if len(files) > 0:
        df = pd.DataFrame([], columns=['filename','event','duration_ratio','avg_acc'])
        for filename in files:
            c += 1
            audio_name = os.path.join(root, filename)
            print(c, audio_name)
            response = !curl -s -X POST {URI} -H  "accept: application/json" -H  "X-Api-Key:{API_KEY}" -H  "Content-Type: multipart/form-data" -F "file=@{audio_name};type=audio/wav"
            try:
                categories_dict = json.loads(response[0])['categories']
                filtered_predictions = filter_predictions(categories_dict, threshold)
                audio_duration = get_audio_duration(audio_name)
                classes_metrics = calculate_metrics(filtered_predictions, audio_duration)
                audio_path = Path(audio_name)
                classes_metrics['filename'] = str(Path(*audio_path.parts[1:]))
                df = pd.concat([df, classes_metrics])
            except Exception as e:
                print(e, audio_name)
                #print(response)
        final_df = df.set_index(['filename', 'event'])
        try:
            with pd.ExcelWriter(f'audios_pedro_{c}.xlsx') as writer:
                final_df.to_excel(writer)
            print(f'Archivo generado: audios_pedro_{c}.xlsx')
        except Exception as e:
            print("No se pudo generar el archivo excel")
            print(e)
            pass

1 audios_pedro/Audios Tarea ACUS155/MAURICIO ANDRES ACEVEDO AGUERO_927620_assignsubmission_file_/PaisajeSonoro5.mp3
2 audios_pedro/Audios Tarea ACUS155/MAURICIO ANDRES ACEVEDO AGUERO_927620_assignsubmission_file_/PaisajeSonoro6.mp3
3 audios_pedro/Audios Tarea ACUS155/MAURICIO ANDRES ACEVEDO AGUERO_927620_assignsubmission_file_/PaisajeSonoro17.mp3
4 audios_pedro/Audios Tarea ACUS155/MAURICIO ANDRES ACEVEDO AGUERO_927620_assignsubmission_file_/PaisajeSonoro12.mp3
5 audios_pedro/Audios Tarea ACUS155/MAURICIO ANDRES ACEVEDO AGUERO_927620_assignsubmission_file_/PaisajeSonoro8.mp3
6 audios_pedro/Audios Tarea ACUS155/MAURICIO ANDRES ACEVEDO AGUERO_927620_assignsubmission_file_/PaisajeSonoro15.mp3
7 audios_pedro/Audios Tarea ACUS155/MAURICIO ANDRES ACEVEDO AGUERO_927620_assignsubmission_file_/PaisajeSonoro3.mp3
8 audios_pedro/Audios Tarea ACUS155/MAURICIO ANDRES ACEVEDO AGUERO_927620_assignsubmission_file_/PaisajeSonoro13.mp3
9 audios_pedro/Audios Tarea ACUS155/MAURICIO ANDRES ACEVEDO AGUERO_9

In [26]:
for audio in wrong_audios:
    os.remove(audio)

In [ ]:
#Total 3.360

In [ ]:
final_df

In [ ]:
final_df = df.set_index(['filename', 'event'])

In [ ]:
with pd.ExcelWriter('audios_pedro.xlsx') as writer:
    final_df.to_excel(writer)